In [1]:
import pandas as pd
from graphviz import Digraph
import numpy as np
from mdp import run as run_mdp
import random

In [2]:
data = {
    "Origin": ["S0", "S1", "S1", "S2"],
    "Action": ["NA", "a", "b", "NA"],
    "S0": [0.1, 0.2, 0.4, 0.3],
    "S1": [0.5, 0.3, 0.4, 0.3],
    "S2": [0.4, 0.5, 0.2, 0.4]
}

# Criando o DataFrame
df = pd.DataFrame(data)

# Exibindo o DataFrame
display(df)


,Origin,Action,S0,S1,S2
0,S0,NA,0.1,0.5,0.4
1,S1,a,0.2,0.3,0.5
2,S1,b,0.4,0.4,0.2
3,S2,NA,0.3,0.3,0.4


In [3]:
# Cria um objeto Graphviz para o gráfico direcionado
dot = Digraph(comment='MDP')

# Ajusta o tamanho do gráfico e configurações de layout
dot.attr(size='15,15')  # Aumenta o tamanho do gráfico
dot.attr('node', shape='circle', width='1', height='1', fontsize='12')  # Ajusta o tamanho dos nós
dot.attr('edge', fontsize='10', style='solid')  # Ajusta estilo das arestas para sólido, aumenta a fonte

# Adiciona os nós (estados)
for state in df['Origin'].unique():
    dot.node(state, state)

# Adiciona as arestas (transições) sem diferenciar por ação
for index, row in df.iterrows():
    origin = row['Origin']
    for dest in ['S0', 'S1', 'S2']:
        if row[dest] > 0:
            # Ação é incluída na label se não for "NA"
            label = f"{row['Action']}, {row[dest]}" if row['Action'] != "NA" else f"{row[dest]}"
            dot.edge(origin, dest, label=label)

# Renderiza o gráfico em um arquivo
dot.render('mdp_graph_updated', format='png', view=True)



'mdp_graph_updated.png'

In [4]:
def random_walk(df, start_state, steps, action='a'):
    current_state = start_state
    path = [current_state]
    
    for _ in range(steps):
        # Filtra as transições possíveis para o estado atual e a ação (incluindo "NA")
        transitions = df[(df['Origin'] == current_state) & ((df['Action'] == action) | (df['Action'] == "NA"))]
        
        # Se não houver transições válidas, termina o loop
        if transitions.empty:
            print("No valid transitions from state", current_state)
            break
        
        # Assegura que estamos selecionando apenas as colunas de estados para as probabilidades
        state_columns = [col for col in df.columns if col.startswith('S')]
        probabilities = transitions[state_columns].values.flatten()
        
        # Verifica se a soma das probabilidades é 1, se não, ajusta
        probabilities = probabilities / probabilities.sum()
        
        # Escolhe o próximo estado baseado nas probabilidades
        next_state = np.random.choice(state_columns, p=probabilities)
        
        path.append(next_state)
        current_state = next_state
    
    return path

In [5]:

# Executa um random walk a partir do estado S0, com 10 passos, e ação 'a'
random_walk_path = random_walk(df, 'S0', 2, 'a')
random_walk_path

['S0', 'S2', 'S1']

In [14]:
# Estados e ações disponíveis
estados = ["S0", "S1", "S2", "S3", "S4", "S5"]
acoes = ["NA", "a", "b", "c"]

# Criando o DataFrame com estados como colunas e uma coluna adicional para ações
df = pd.DataFrame(columns=["Origin", "Action"] + estados)

# Adicionando as linhas ao DataFrame
for estado_origem in estados:
    # Decidindo aleatoriamente as ações disponíveis para cada estado, exceto S0
    # Garantindo que pelo menos uma ação seja selecionada e sem repetição
    acoes_disponiveis = np.random.choice(acoes[1:], np.random.randint(1, 4), replace=False).tolist()

    for acao in acoes_disponiveis:
        # Gerando probabilidades aleatórias para cada estado de chegada
        probabilidades = np.random.dirichlet(np.ones(len(estados)), size=1)[0]
        # Adicionando estado de origem, ação e probabilidades ao DataFrame
        linha = [estado_origem, acao] + probabilidades.tolist()
        df.loc[len(df)] = linha

# Ajustando o index para melhor visualização
df.reset_index(drop=True, inplace=True)

display(df)

,Origin,Action,S0,S1,S2,S3,S4,S5
0,S0,c,0.615217,0.082413,0.038262,0.066272,0.018329,0.179507
1,S0,a,0.110037,0.156031,0.224684,0.431884,0.056095,0.021269
2,S0,b,0.142797,0.112666,0.041958,0.176912,0.448041,0.077626
3,S1,c,0.090028,0.206510,0.080623,0.199455,0.193839,0.229544
4,S1,b,0.024891,0.015106,0.425937,0.275687,0.098958,0.159421
5,S1,a,0.151172,0.282102,0.149877,0.062888,0.346672,0.007288
6,S2,b,0.216236,0.185744,0.190787,0.041814,0.239849,0.125570
7,S2,a,0.227206,0.289561,0.200258,0.053532,0.111051,0.118393
8,S2,c,0.016132,0.067885,0.202658,0.039738,0.005062,0.668525
9,S3,b,0.007093,0.150897,0.464837,0.026689,0.021248,0.329236


In [2]:
printer = run_mdp(path = "correct_ex.mdp", return_printer=True)

States: ['S0', 'S1', 'S2']
Actions: ['a', 'b', 'c', 'd']
Transition from S0 with no action and targets ['S1', 'S2'] with weights [5, 5]
Transition from S1 with action b and targets ['S1', 'S0'] with weights [2, 8]
Transition from S1 with action a and targets ['S2', 'S0', 'S1'] with weights [1, 3, 6]
Transition from S2 with action c and targets ['S0', 'S1'] with weights [5, 5]
Transition from S2 with action d and targets ['S0'] with weights [5]


In [3]:
df = printer.transactions_prob
print(printer.declared_actions)
print(printer.declared_states)


print(type(printer.declared_actions))

display(df)

{'d', 'b', 'c', 'a'}
{'S1', 'S0', 'S2'}
<class 'set'>


,Origin,Action,S1,S0,S2
0,S0,NA,0.5,0.0,0.5
1,S1,b,0.2,0.8,0.0
2,S1,a,0.6,0.3,0.1
3,S2,c,0.5,0.5,0.0
4,S2,d,0.0,1.0,0.0


Dia 16/02/2024

In [4]:
def gerar_preferencias_acoes(estados, acoes, modo="input"):
    preferencias = {}

    for estado in estados:
        if modo == "input":
            print(f"\nEstado atual: {estado}")
            print("Ações disponíveis: " + ", ".join(acoes))
            preferencia_str = input(f"Digite as ações em ordem de preferência para o estado {estado} (separadas por vírgula): ")
            preferencia_lista = [acao.strip() for acao in preferencia_str.split(",") if acao.strip() in acoes]
            preferencias[estado] = preferencia_lista

        elif modo == "random":
            acoes_aleatorias = list(acoes)
            random.shuffle(acoes_aleatorias)
            preferencias[estado] = acoes_aleatorias
            
        else:
            print("Modo desconhecido. Por favor, escolha 'input' ou 'random'.")
            return {}

    return preferencias

In [5]:
preferencias = gerar_preferencias_acoes(printer.declared_states, printer.declared_actions, modo="input")
print(preferencias)


Estado atual: S1
Ações disponíveis: d, b, c, a

Estado atual: S0
Ações disponíveis: d, b, c, a

Estado atual: S2
Ações disponíveis: d, b, c, a
{'S1': [], 'S0': ['a'], 'S2': ['b', 'c', 'd']}


In [41]:
def simular_random_walk(p, num_transitions):
    # "p" is a printer object
    
    df = p.transactions_prob
    preferencias = gerar_preferencias_acoes(p.declared_states, p.declared_actions, modo="random")
    print("Preferências de ações (modo random):", preferencias, "\n")

    estado_atual = random.choice(list(p.declared_states))  #TODO : mudar isso aqui para pegar o primeiro do REMI.

    caminho = estado_atual  # Iniciar o registro do caminho com o estado inicial
    probabilidade_acumulada = 1

    print(f"Estado inicial: {estado_atual}")

    for _ in range(num_transitions):
        df_estado_atual = df[df['Origin'] == estado_atual]
        acao_selecionada = None
        probabilidade_escolhida = None

        if df_estado_atual.iloc[0]['Action'] == "NA":
            probabilidades = df_estado_atual.iloc[0, 2:].astype(float).values
            acao_selecionada = "NA"
        else:
            for acao_preferida in preferencias[estado_atual]:
                df_acao_preferida = df_estado_atual[df_estado_atual['Action'] == acao_preferida]
                if not df_acao_preferida.empty:
                    probabilidades = df_acao_preferida.iloc[0, 2:].astype(float).values
                    acao_selecionada = acao_preferida
                    break

        probabilidades = probabilidades / np.sum(probabilidades)
        estados_possiveis = df_estado_atual.columns[2:]
        proximo_estado = np.random.choice(estados_possiveis, p=probabilidades)
        probabilidade_escolhida = probabilidades[np.where(estados_possiveis == proximo_estado)[0][0]]
        
        probabilidade_acumulada *= probabilidade_escolhida
        estado_passado = estado_atual
        estado_atual = proximo_estado
        caminho += f" -> {estado_atual}"  # Atualizar o caminho

        print(f"Origin: {estado_passado}, Action: {acao_selecionada}, Destiny: {estado_atual}, Prob of next step: {probabilidade_escolhida:.3f}, Prob path up to here: {probabilidade_acumulada:.5f}, Path: {caminho}," + "\n")

    print(f"Complete Path: {caminho}")

In [42]:
simular_random_walk(printer, num_transitions = 5)

Preferências de ações (modo random): {'S1': ['a', 'c', 'b', 'd'], 'S0': ['d', 'c', 'b', 'a'], 'S2': ['c', 'd', 'b', 'a']} 

Estado inicial: S1
Origin: S1, Action: a, Destiny: S0, Prob of next step: 0.300, Prob path up to here: 0.30000, Path: S1 -> S0,

Origin: S0, Action: NA, Destiny: S1, Prob of next step: 0.500, Prob path up to here: 0.15000, Path: S1 -> S0 -> S1,

Origin: S1, Action: a, Destiny: S1, Prob of next step: 0.600, Prob path up to here: 0.09000, Path: S1 -> S0 -> S1 -> S1,

Origin: S1, Action: a, Destiny: S0, Prob of next step: 0.300, Prob path up to here: 0.02700, Path: S1 -> S0 -> S1 -> S1 -> S0,

Origin: S0, Action: NA, Destiny: S1, Prob of next step: 0.500, Prob path up to here: 0.01350, Path: S1 -> S0 -> S1 -> S1 -> S0 -> S1,

Complete Path: S1 -> S0 -> S1 -> S1 -> S0 -> S1


In [23]:
display(printer.transactions_prob)

,Origin,Action,S1,S0,S2
0,S0,NA,0.5,0.0,0.5
1,S1,b,0.2,0.8,0.0
2,S1,a,0.6,0.3,0.1
3,S2,c,0.5,0.5,0.0
4,S2,d,0.0,1.0,0.0
